In [1]:
import pandas as pd
import json
import urllib
from urllib.request import Request, urlopen
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [2]:
total = pd.read_csv('./강남_total_detail.csv')

In [3]:
total.drop(['Unnamed: 0'], axis=1, inplace=True)
total.index = total.index+1
total

,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점,lat,lng
1,김북순큰남비집,서울특별시 강남구 압구정로2길 15 신사아이빌 1F,한정식 / 백반 / 정통 한식,만원 미만,25,4.0,37.520699,127.019975
2,두부공작소,서울특별시 강남구 강남대로160길 21,한정식 / 백반 / 정통 한식,만원 미만,16,3.8,37.519766,127.020477
3,딸부자네불백,서울특별시 강남구 강남대로158길 21,고기 요리,만원 미만,15,3.8,37.519231,127.020512
4,빌라드스파이시,서울특별시 강남구 도산대로13길 14-3,기타 한식,만원 미만,38,3.6,37.519027,127.023549
5,목포집,서울특별시 강남구 강남대로152길 13,닭 / 오리 요리,만원 미만,11,3.6,37.517342,127.020489
...,...,...,...,...,...,...,...,...
17873,마토,서울특별시 강남구 선릉로146길 23 4F,칵테일 / 와인,4만원 이상,1,NaN,NaN,NaN
17874,레 끌레 드 크리스탈,서울특별시 강남구 압구정로79길 8 하나빌딩 2-3F,칵테일 / 와인,4만원 이상,2,NaN,NaN,NaN
17875,더부즈,서울특별시 강남구 도산대로85길 13 B1,칵테일 / 와인,3만원-4만원,1,NaN,NaN,NaN
17876,클래시크,서울특별시 강남구 압구정로79길 81 B1,칵테일 / 와인,4만원 이상,1,NaN,NaN,NaN


In [4]:
start = '서울특별시 강남구 강남대로 396' ## 강남역
start_name = '현재위치'

In [5]:
first = total['가게 주소'][50]
second = total['가게 주소'][1235]
goal = total['가게 주소'][1787]

In [6]:
client_id = '5i37rrbzjo'
client_secret = '51wepLcxNeo22EIRAsK7TSmPkGUjKBw4gjgz1Pmk'

In [7]:
def get_location(loc) :

    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    + urllib.parse.quote(loc)

    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urlopen(request)
    res = response.getcode()

    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return한다
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        #print(response_body)
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 :
            # 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lng = response_body['addresses'][0]['x']
            return [lng, lat]
        else :
            print('location not exist')

    else :
        print('ERROR')

In [8]:
def get_name(loc):
    if loc == '서울특별시 강남구 강남대로 396':
        return '현재위치'
    else:
        first_name = total[(total['가게 주소']==loc)]['가게이름']
        return first_name.item()

In [9]:
def get_color(loc):
    if loc == start:
        color = 'red'
    elif loc == first:
        color = 'blue'
    elif loc == second:
        color = 'green'
    return color

In [10]:
## 개별 길찾기
def get_optimal_route(start, goal):
    # *-- Directions 5 활용 코드 --*

    start = start
    goal = goal
    start_gps = get_location(start)
    goal_gps = get_location(goal)
    start_name = get_name(start)
    goal_name = get_name(goal)
    color = get_color(start)
    data = []
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={},{}&goal={},{}&option=trafast'.format(start_gps[0], start_gps[1], goal_gps[0], goal_gps[1])
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if (res == 200) :
        response_body = response.read().decode('utf-8')
        data.append(json.loads(response_body))

    else :
        print('ERROR')
        
    taxiFare = '택시비 예상금액'+' '+ '약 '+ str(data[0]['route']['trafast'][0]['summary']['taxiFare']) + '원'
    distance = '예상거리 ' + str(data[0]['route']['trafast'][0]['summary']['distance']) + 'm'
    mid = len(data[0]['route']['trafast'][0]['path'])
    lat_center = data[0]['route']['trafast'][0]['path'][int(mid/2)][1]
    lng_center = data[0]['route']['trafast'][0]['path'][int(mid/2)][0]
    guide_comment = str(start_name) + ' ' + '에서' + ' ' + str(goal_name) + ' ' + '까지' 
    
    center = [lat_center, lng_center]
    loc=[]
    m = folium.Map(location=center, zoom_start=14)
    
    folium.Marker(
        location=[start_gps[1], start_gps[0]],
        popup='{}'.format(start_name)
    ).add_to(m)
    
    folium.Marker(
        location=[goal_gps[1], goal_gps[0]],
        popup='{}'.format(goal_name)
    ).add_to(m)
    
    for idx in data[0]['route']['trafast'][0]['path']:
        loc.append([idx[1], idx[0]])
    folium.PolyLine(locations=loc, tooltip='Polyline', color= color).add_to(m)
    
    print(guide_comment)
    print(distance)
    print(taxiFare)
    print(' ')
    return m

In [11]:
## 종합 길찾기
def get_optimal_route(start, first, second, goal) :
    # *-- Directions 5 활용 코드 --*

    start = start
    first = first
    start_gps = get_location(start)
    goal_gps = get_location(first)
    start_name = get_name(start)
    goal_name = get_name(first)

    
    data = []
    
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={},{}&goal={},{}&option=trafast'.format(start_gps[0], start_gps[1], goal_gps[0], goal_gps[1])
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if (res == 200) :
        response_body = response.read().decode('utf-8')
        data.append(json.loads(response_body))

    else :
        print('ERROR')
        
    taxiFare = int(data[0]['route']['trafast'][0]['summary']['taxiFare'])
    distance = '예상거리 ' + str(data[0]['route']['trafast'][0]['summary']['distance']) + 'm'
    mid = len(data[0]['route']['trafast'][0]['path'])
    lat_center = data[0]['route']['trafast'][0]['path'][int(mid/2)][1]
    lng_center = data[0]['route']['trafast'][0]['path'][int(mid/2)][0]
    guide_comment = str(start_name) + ' ' + '에서' + ' ' + str(goal_name) + ' ' + '까지' 
    
    center = [lat_center, lng_center]
    loc=[]
    loc1=[]
    loc2=[]
    m = folium.Map(location=center, zoom_start=14)
        
    folium.Marker(
        location=[start_gps[1], start_gps[0]],
        popup='<pre>{}<pre>'.format(start_name),
        icon= folium.Icon(color='red')
    ).add_to(m)


    for idx in data[0]['route']['trafast'][0]['path']:
        loc.append([idx[1], idx[0]])
    folium.PolyLine(locations=loc, tooltip='Polyline', color= 'red').add_to(m)
    
    print(guide_comment)
    print(distance)
    print('택시비 예상금액'+' '+ '약 '+ str(taxiFare) + '원')
    print(' ')

    ############################
    first = first
    second = second
    start_gps = get_location(first)
    goal_gps = get_location(second)
    start_name = get_name(first)
    goal_name = get_name(second)
    color = get_color(first)
    
    
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={},{}&goal={},{}&option=trafast'.format(start_gps[0], start_gps[1], goal_gps[0], goal_gps[1])
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if (res == 200) :
        response_body = response.read().decode('utf-8')
        data.append(json.loads(response_body))

    else :
        print('ERROR')
        
    taxiFare1 = int(data[1]['route']['trafast'][0]['summary']['taxiFare'])
    distance = '예상거리 ' + str(data[1]['route']['trafast'][0]['summary']['distance']) + 'm'
    mid = len(data[1]['route']['trafast'][0]['path'])
    guide_comment = str(start_name) + ' ' + '에서' + ' ' + str(goal_name) + ' ' + '까지' 

    
    folium.Marker(
        location=[start_gps[1], start_gps[0]],
        popup='<pre>{}<pre>'.format('첫번째 가게: ' + start_name),
        icon= folium.Icon(color='blue')
    ).add_to(m)
    

    for idx in data[1]['route']['trafast'][0]['path']:
        loc1.append([idx[1], idx[0]])
    folium.PolyLine(locations=loc1, tooltip='Polyline', color= 'blue').add_to(m)
    
    print(guide_comment)
    print(distance)
    print('택시비 예상금액'+' '+ '약 '+ str(taxiFare1) + '원')
    print(' ')
    
    ###########
    
    second = second
    goal = goal
    start_gps = get_location(second)
    goal_gps = get_location(goal)
    start_name = get_name(second)
    goal_name = get_name(goal)
    color = get_color(second)
    
    
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={},{}&goal={},{}&option=trafast'.format(start_gps[0], start_gps[1], goal_gps[0], goal_gps[1])
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if (res == 200) :
        response_body = response.read().decode('utf-8')
        data.append(json.loads(response_body))

    else :
        print('ERROR')
        
    taxiFare2 = int(data[2]['route']['trafast'][0]['summary']['taxiFare'])
    distance = '예상거리 ' + str(data[2]['route']['trafast'][0]['summary']['distance']) + 'm'
    mid = len(data[2]['route']['trafast'][0]['path'])
    guide_comment = str(start_name) + ' ' + '에서' + ' ' + str(goal_name) + ' ' + '까지' 

    
    folium.Marker(
        location=[start_gps[1], start_gps[0]],
        popup='<pre>{}<pre>'.format('두번째 가게: ' + start_name),
        icon= folium.Icon(color='green')
    ).add_to(m)
    
    folium.Marker(
        location=[goal_gps[1], goal_gps[0]],
        popup='<pre>{}<pre>'.format('마지막 가게: ' + goal_name),
        icon= folium.Icon(color='green')
    ).add_to(m)

    for idx in data[2]['route']['trafast'][0]['path']:
        loc2.append([idx[1], idx[0]])
    folium.PolyLine(locations=loc2, tooltip='Polyline', color= 'green').add_to(m)
    
    print(guide_comment)
    print(distance)
    print('택시비 예상금액'+' '+ '약 '+ str(taxiFare2) + '원')
    #print('총합 택시비 약 ' + str(taxiFare + taxiFare1 + taxiFare2) + '원')
    print(' ')
    
    return m

In [12]:
get_optimal_route(start, first, second, goal)

현재위치 에서 신사동가자 까지
예상거리 4092m
택시비 예상금액 약 6570원
 
신사동가자 에서 에슬로우 까지
예상거리 3875m
택시비 예상금액 약 6250원
 
에슬로우 에서 모리식당 까지
예상거리 13620m
택시비 예상금액 약 15690원
 
